In [41]:
import math
import geopy.distance
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim

%matplotlib inline

train_data = pd.read_csv('data/stores_train.csv')
test_data = pd.read_csv('data/stores_test.csv')
busstops = pd.read_csv('data/busstops_norway.csv')
grunnkrets_age = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('data/grunnkrets_income_households.csv')
grunnkrets_stripped = pd.read_csv('data/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')

In [42]:
def dist_to_all_km(lat, lon, df):

    # coordinates in radians
    lat1 = lat*math.pi/180
    lon1 = lon*math.pi/180
    lat2 = df['lat']*math.pi/180 # go through whole lat column
    lon2 = df['lon']*math.pi/180 # go through whole lon column

    # store original coordinates in new dataframe
    distances = pd.DataFrame()
    distances['lat'] = df['lat'].copy()
    distances['lon'] = df['lon'].copy()

    # calculate cartesian coordinates
    R = 6371 # Earth radius in km
    df['x'] = R*np.cos(lat2)*np.cos(lon2)
    df['y'] = R*np.cos(lat2)*np.sin(lon2)
    df['z'] = R*np.sin(lat2)
    x1 = R*np.cos(lat1)*np.cos(lon1)
    y1 = R*np.cos(lat1)*np.sin(lon1)
    z1 = R*np.sin(lat1)

    # calculate distance, store as new column in the distances dataframe
    distances['dist'] = np.sqrt(np.square(df['x']-x1)+np.square(df['y']-y1)+np.square(df['z']-z1))

    return distances['dist'].squeeze()

In [43]:
df = pd.DataFrame(train_data, columns=['store_id', 'lat', 'lon', 'grunnkrets_id'])
df = pd.merge(df, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')
df = df.drop_duplicates(subset=['store_id'], keep='first').reset_index(drop=True)

# df.isnull().sum() # 30 NaNs in municipality_name

index_of_NaN, idy = np.where(pd.isnull(df))

for i in index_of_NaN:
    lat = df._get_value(i,'lat')
    lon = df._get_value(i,'lon')
    dist_to_all = dist_to_all_km(lat, lon, df).to_frame()
    dist_to_all = dist_to_all.sort_values('dist').reset_index()
    dist_to_all = dist_to_all.rename(columns={'index':'index_in_train_data'})

    # Find index of closest store that isn't itself and that has a grunnkrets_id
    index_of_closest_store = 0
    while dist_to_all.iloc[index_of_closest_store]['dist'] == 0 or pd.isna(df.at[dist_to_all.iloc[index_of_closest_store]['index_in_train_data'], 'municipality_name']):
        index_of_closest_store = index_of_closest_store + 1
    
    # Set municipality_name as the municipality name of the closest store that has one
    df.at[i,'municipality_name'] = df.at[dist_to_all.iloc[index_of_closest_store]['index_in_train_data'], 'municipality_name']
